In [4]:
# Regarding mean and std: https://stackoverflow.com/questions/57532661/how-do-they-know-mean-and-std-the-input-value-of-transforms-normalize
# mean and std calculation: https://discuss.pytorch.org/t/about-normalization-using-pre-trained-vgg16-networks/23560/5

import torchvision.models as models
import torch
import torch.nn as nn
import torch.nn.functional as F
import glob
import numpy as np
import os
import json
from torch.utils import data
from torchvision.datasets.folder import pil_loader
import matplotlib.pyplot as plot
from torchvision import transforms
import torch.optim as optim
from tqdm import tqdm
import matplotlib.pyplot as plt 
from utils import load_json
import timeit
import importlib.util
spec = importlib.util.spec_from_file_location("module.name", "/home/arnab/Desktop/dnn_offloading/Algorithms/NiN.py")
NiN = importlib.util.module_from_spec(spec)
spec.loader.exec_module(NiN)

# define pytorch device - useful for device-agnostic execution
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device: " + str(device))
#DEVICE_IDS = 0
# Global
Model = None
all_loss = []
epoch_num = []
class BDDDataset(data.Dataset):

    def __init__(self, root, train, transform=None):
        self.root = root
        self.train = train
        self.transform = transform
        self.samples = None
        self.image_namelist_ = None
        self.img_label = None
        self.CATEGORY = None
        self.LABELS = None
        self.category_label()
        
        if self.train:
            self.image_namelist()
        else:
            self.image_namelist_test()
        
        self.prepare_data_by_labels()
        
    def category_label(self):
        self.CATEGORY = ['rider', 'traffic light', 'lane', 'traffic sign', 'bike', 'motor', 'truck', 'bus', 'car', 'drivable area', 'person', 'train']
        self.LABELS={}
        for i,key in enumerate(self.CATEGORY):
            self.LABELS.update({key : i})
    # train imageList  
    def image_namelist(self):
        train_label_path = "/home/arnab/Desktop/Data/labels/bdd1k_labels_images_train.json"
        self.img_label = load_json(train_label_path)
        self.image_namelist_ = []
        for img in self.img_label:
            #print("Name: {}".format(img['name']))
            self.image_namelist_.append(img['name'])
            
    # test imageList
    def image_namelist_test(self):
        test_label_path = "/home/arnab/Desktop/Data/labels/bdd1k_labels_images_test.json"
        self.img_label = load_json(test_label_path)
        self.image_namelist_ = []
        for img in self.img_label:
            #print("Name: {}".format(img['name']))
            self.image_namelist_.append(img['name'])

    # (image->image_stat(label)) is considered
    def prepare_data_by_labels(self):
        self.samples = []
        if self.train:
            image_files = glob.glob(
                os.path.join(self.root, 'train/*.jpg'))
            image_dir = os.path.join(self.root, 'train')
        else:
            image_files = glob.glob(
                os.path.join(self.root, 'test/*.jpg'))
            image_dir = os.path.join(self.root, 'test')
        
        for image_file in self.image_namelist_:
            image_path = os.path.join(image_dir,image_file)
            for img_stat in self.img_label:
                if img_stat['name'] == image_file:
                    if os.path.exists(image_path):
                        categories_list = []
                        categories_list_bool = [0] * len(self.CATEGORY)
                        for l in img_stat['labels']:
                            categories_list.append(l['category'])
                            categories_list = list(set(categories_list))
                        for cat in categories_list:
                            categories_list_bool[self.LABELS[cat]] = 1
                        
                        self.samples.append([image_path, torch.Tensor(categories_list_bool)])
                    else:
                        raise FileNotFoundError
    
    def __getitem__(self, index):
        
        image_path, img_stat = self.samples[index]

        image = pil_loader(image_path)

        if self.transform is not None:
            image = self.transform(image)

        return image, img_stat
    

    def __len__(self):
        return len(self.samples)
    


# Training the model
def Train(model,epoches):
    global all_loss,epoch_num
    NUM_CLASSES = 12
    BATCH_SIZE = 128
    EPOCHS = int(epoches)
    iterator = 1
    
    net, transform = Model_Infomation(NUM_CLASSES, model)
    loader = data.DataLoader(
        BDDDataset('/home/arnab/Desktop/Data/images', train=True, transform=transform),
        batch_size=BATCH_SIZE,
        shuffle=True)
    
    """
    Each image data representation: 
    torch.Size([1, 3, 227, 227])
    tensor([[0., 1., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0.]]) Categories of objects in the image.
    """
    
    # Optimizer and loss function
    optimizer = optim.Adam(net.parameters(), lr=0.001)
    #optimizer = optim.AdamW(net.parameters(), lr=0.001)
    #loss_function = nn.MSELoss()
    #loss_function = nn.MultiLabelSoftMarginLoss()
    loss_function = nn.BCEWithLogitsLoss()
    
    for epoch in range(EPOCHS):
        print("Epoch: {}/{}\n".format(epoch+1,EPOCHS))
        for img,label in loader:
            if iterator == 1:
                print("Image size: " + str(img.shape))
                print("Label size: " + str(label.shape))
            img,label = img.to(device), label.to(device)
            net.zero_grad()
            output = net(img)
            loss = loss_function(output, label)
            loss.backward()
            optimizer.step()
            iterator += 1
        epoch_num.append(epoch+1)
        all_loss.append(loss.item())
        print("Loss: " + str(loss))
        print("Total Iterations: " + str(iterator-1))
    torch.save(net.state_dict(),"/home/arnab/Desktop/Data/NiN_EPOCH_1_trained_model.pt")
        
# Calculating accuracy
def Accuracy(model):
    correct = 0
    total = 0
    NUM_CLASSES = 12
    BATCH_SIZE = 1
    
    net, transform = Model_Infomation(NUM_CLASSES, model)
    net.load_state_dict(torch.load("/l/Data/dnn_parameters/trained_model.pt"))
    net.eval()
    
    test_loader = data.DataLoader(
        BDDDataset('/l/Data/images', train=False, transform=transform),
        batch_size=BATCH_SIZE,
        shuffle=True)
    
    test_X = []
    test_y = []
    for X,y in test_loader:
        test_X.append(X)
        test_y.append(y)
    hamming = []
    with torch.no_grad():
        for i in tqdm(range(len(test_X))):
            X = test_X[i].to(device)
            y = test_y[i]
            real_class = y #torch.argmax(test_y[i])
            net_out = net(X)#[0]  # returns a list, 
            predicted_class = net_out.cpu()#torch.argmax(net_out)
            #predicted_class = torch.sigmoid(predicted_class).data > 0.5
            a = nn.ReLU()
            predicted_class = a(predicted_class).data > 0
            predicted_class = predicted_class.double()
            
            # convert from tensor to numpy array
            real_class = real_class.numpy()[:]
            predicted_class = predicted_class.numpy()[:]

            # calculate incorrect classification (n_c)
            n_c = np.count_nonzero(real_class!=predicted_class)
            #print(n_c)
            hamming.append(n_c)
            if n_c <= 2:
                correct += 1
            total += 1
    print("Accuracy: ", round(correct/total, 3)*100)
    
    
def Model_Infomation(NoC,model):
    IMAGE_DIM = 0
    net = None
    if model == "1":
        IMAGE_DIM = 227
        #net = models.alexnet(num_classes=NoC)
        net = models.alexnet(num_classes=NoC).to(device)
        print("Model: AlexNet")

    elif model == "2":
        IMAGE_DIM = 224
        #net = models.resnet34(num_classes=NoC)
        net = models.resnet34(num_classes=NoC).to(device)
        print("Model: ResNet")

    elif model == "3":
        IMAGE_DIM = 224
        #net = models.vgg16(num_classes=NoC)
        net = models.vgg16(num_classes=NoC).to(device)
        print("Model: VGG16")
        
    elif model == "4":
        IMAGE_DIM = 32
        net = NiN.NIN(num_classes=NoC)
        #net = NiN.NIN(num_classes=NoC).to(device)
        print("Model: NiN")
        
    else:
        IMAGE_DIM = 227
        #net = models.alexnet(num_classes=NoC)
        net = models.alexnet(num_classes=NoC).to(device)
        print("Model: AlexNet")
        
    
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(IMAGE_DIM),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),]) # ImageNet dataset normalization
    
    return net,transform

def plot_loss():
    global epoch_num,all_loss
    plt.plot(epoch_num, all_loss)
    # naming the x axis 
    plt.xlabel('Epoch Number')
    # naming the y axis 
    plt.ylabel('Losses')
    
def main():
    
    # Training 
    # 1: AlexNet 
    # 2: ResNet34 
    # 3: VGG16
    # 4: NiN
    model = input("Enter Model Number [1: AlexNet 2: ResNet34 3: VGG16 4: NiN]: ")
    epoches = input("Number of Epoches: ")
    start = timeit.default_timer()
    Train(model,epoches)
    stop = timeit.default_timer()
    print('Time: ', (stop - start)/60)
    
    # Accuracy
    #Accuracy(model)
    
    # Plot
    #plot_loss()
    '''
    for param_tensor in net.state_dict():
        print(param_tensor, "\t", net.state_dict()[param_tensor].size())
    '''
      
if __name__ == '__main__':
    main()

Device: cpu
Enter Model Number [1: AlexNet 2: ResNet34 3: VGG16 4: NiN]: 4
Number of Epoches: 5
Model: NiN
Epoch: 1/5

Image size: torch.Size([128, 3, 32, 32])
Label size: torch.Size([128, 12])
Loss: tensor(0.5854, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
Total Iterations: 8
Epoch: 2/5

Loss: tensor(0.5265, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
Total Iterations: 16
Epoch: 3/5

Loss: tensor(0.5443, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
Total Iterations: 24
Epoch: 4/5

Loss: tensor(0.5407, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
Total Iterations: 32
Epoch: 5/5

Loss: tensor(0.5450, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
Total Iterations: 40
Time:  1.6391909223833256
